In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import talib as ta
import os
import mplfinance as mpf
import numpy as np

In [2]:
def get_time_stamp():
    return datetime.now().strftime('%Y%m%d%H%M%S')

In [3]:
directory = 'indodax-' + get_time_stamp()
os.makedirs(directory, exist_ok=True)

In [4]:
url = 'https://indodax.com'

In [5]:
def get_pairs() :
    return requests.get(url + '/api/pairs').json()

In [6]:
def get_ticker(pairid, timeframe, timefrom, timeto) :
    return requests.get(url + '/tradingview/history_v2?from=' + str(timefrom) + '&symbol=' + pairid +'&tf=' + timeframe + '&to=' + str(timeto)).json()

In [7]:
def resampling_weekly(df_daily) :
    return df_daily.resample('W').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

In [8]:
def show_candlestick(tickername, tickerbase, df) :
    df.set_index('Date', inplace=True)
    mpf.plot(df, type='candle', style='binance', volume=True, title=f'Pergerakan Harga {tickername} terhadap {tickerbase}',
         savefig=dict(fname=os.path.join(directory, f'Candlestick-{tickername}-{tickerbase}-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight'))

    if len(df['Close']) >= 100 :
        df_weekly = resampling_weekly(df)
        mpf.plot(df_weekly, type='candle', style='binance', volume=True, title=f'Pergerakan Harga Mingguan {tickername} terhadap {tickerbase}',
            savefig=dict(fname=os.path.join(directory, f'Candlestick-Weekly-{tickername}-{tickerbase}-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight'))


In [9]:
def show_chart(df, tickername, tickerbase) :
    time_now = datetime.now().strftime('%d-%m-%Y %H:%M:%S')

    # Membuat figure dan dua subplot yang berbagi sumbu x
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(14, 10))

    # Pergerakan Harga
    ax1.plot(df['Date'], df['Close'], label=f'{tickername}/{tickerbase}', color='blue')
    # ax1.plot(df['Date'], df['EMA_13'], linewidth=0.5, label='EMA 13', color='yellow')
    # ax1.plot(df['Date'], df['EMA_21'], linewidth=0.5, label='EMA 21', color='orange')
    ax1.set_title(f'Pergerakan Harga {tickername} terhadap {tickerbase}')
    ax1.set_ylabel(f'Harga ({tickerbase})')
    ax1.legend()
    ax1.grid(True)
    ax1.text(0.99, 1.05, time_now,
         transform=ax1.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax1.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax1.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # RSI
    ax2.plot(df['Date'], df['RSI_14'], label=f'RSI {tickername}/{tickerbase}', color='orange')
    ax2.axhline(70, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax2.axhline(30, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax2.set_title(f'Grafik RSI {tickername} terhadap {tickerbase}')
    # ax2.set_xlabel('Tanggal')
    ax2.set_ylabel('RSI')
    ax2.legend()
    ax2.grid(True)
    ax2.text(0.99, 1.05, time_now,
         transform=ax2.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax2.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax2.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Stochastic
    ax3.plot(df['Date'], df['Stoch_%K'], label='%K', color='blue')
    ax3.plot(df['Date'], df['Stoch_%D'], label='%D', color='orange')
    ax3.axhline(80, linestyle='--', alpha=0.5, color='red')  # Garis overbought
    ax3.axhline(20, linestyle='--', alpha=0.5, color='green')  # Garis oversold
    ax3.set_title(f'Grafik Stochastic {tickername} terhadap {tickerbase}')
    # ax3.set_xlabel('Tanggal')
    ax3.set_ylabel('Nilai')
    ax3.legend()
    ax3.grid(True)
    ax3.text(0.99, 1.05, time_now,
         transform=ax3.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax3.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax3.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # OBV
    ax4.plot(df['Date'], df['OBV'], label='OBV', color='blue')
    ax4.set_title(f'Grafik OBV {tickername}')
    ax4.set_xlabel('Tanggal')
    ax4.set_ylabel('Nilai')
    ax4.legend()
    ax4.grid(True)
    ax4.text(0.99, 1.05, time_now,
         transform=ax4.transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    ax4.text(0.5, 0.5, 'Generated by Yoga',
         transform=ax4.transAxes,
         fontsize=20, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)

    # Menyesuaikan tata letak agar tidak saling tumpang tindih
    plt.tight_layout()
    # plt.show()
    plt.savefig(os.path.join(directory, 'Chart-' + tickername + '-' + tickerbase + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

    show_candlestick(tickername, tickerbase, df)

In [10]:
def get_df_from_ticker(ticker_name, timeframe, timefrom, timeto) :
    ticker_name = get_ticker(ticker_name + 'IDR', timeframe, timefrom, timeto)
    df_1 = pd.DataFrame(ticker_name)
    if df_1.empty :
        return df_1
    df_1['Date'] = pd.to_datetime(df_1['Time'], unit='s')
    df_1 = df_1.sort_values(by='Date')
    return df_1

In [11]:
def check_first_candle(df) :
    first_high = df['High'].iloc[0]
    first_low = df['Low'].iloc[0]
    if first_high/first_low > 2 :
        return True
    else :
        return False

In [12]:
def get_history_btc_pair(tickername, df_coin_idr, dfbtcidr) :
    ticker_name_low = tickername.lower()
    df_merged = pd.merge(df_coin_idr, dfbtcidr, on='Date', suffixes=('_'+ticker_name_low, '_btc'))
    df_merged['Open_' + ticker_name_low +'_btc'] = df_merged['Open_' + ticker_name_low] / df_merged['Open_btc']
    df_merged['High_' + ticker_name_low +'_btc'] = df_merged['High_' + ticker_name_low] / df_merged['High_btc']
    df_merged['Low_' + ticker_name_low + '_btc'] = df_merged['Low_' + ticker_name_low] / df_merged['Low_btc']
    df_merged['Close_'+ ticker_name_low +'_btc'] = df_merged['Close_' + ticker_name_low] / df_merged['Close_btc']
    df_merged['Volume_'+ ticker_name_low +'_btc'] = df_merged['Volume_' + ticker_name_low]

    df_final = df_merged[['Date', 'Open_' + ticker_name_low +'_btc', 'High_' + ticker_name_low +'_btc', 'Low_' + ticker_name_low + '_btc', 'Close_' + ticker_name_low +'_btc', 'Volume_' + ticker_name_low +'_btc']].copy()
    df_final.rename(columns={
        'Open_' + ticker_name_low +'_btc': 'Open',
        'High_' + ticker_name_low +'_btc': 'High',
        'Low_' + ticker_name_low + '_btc': 'Low',
        'Close_' + ticker_name_low +'_btc': 'Close',
        'Volume_' + ticker_name_low +'_btc': 'Volume'
    }, inplace=True)

    mask = df_final['High'] < df_final['Low']
    df_final.loc[mask, ['High', 'Low']] = df_final.loc[mask, ['Low', 'High']].values

    if check_first_candle(df_final) :
        df_final.drop(index=0, inplace=True)

    df_final['Volume'] = pd.to_numeric(df_final['Volume'])

    df_final['Return'] = df_final['Close'].pct_change()
    df_final['Cum Return'] = (1 + df_final['Return']).cumprod() - 1

    df_final['RSI_14'] = ta.RSI(df_final['Close'], timeperiod=14)

    df_final['OBV'] = ta.OBV(df_final['Close'], df_final['Volume'])

    slowk, slowd = ta.STOCH(df_final['High'], df_final['Low'], df_final['Close'],
                           fastk_period=5,
                           slowk_period=3,
                           slowd_period=3)
    df_final['Stoch_%K'] = slowk
    df_final['Stoch_%D'] = slowd

    len_data = len(df_merged['Close_' + ticker_name_low])
    if len_data > 2 :
        min_time_period = min(len_data-1, 90)
        df_final['Beta'] = ta.BETA(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)
        df_final['Correl'] = ta.CORREL(df_merged['Close_' + ticker_name_low], df_merged['Close_btc'], timeperiod=min_time_period)

    df_final['Change1D'] = df_final['Close'].pct_change() * 100
    df_final['Change7D'] = df_final['Close'].pct_change(periods=7) * 100
    df_final['Change30D'] = df_final['Close'].pct_change(periods=30) * 100

    # df_final['EMA_13'] = df_final['Close'].ewm(span=13, adjust=False).mean()
    # df_final['EMA_21'] = df_final['Close'].ewm(span=21, adjust=False).mean()

    return df_final

In [13]:
pairs = get_pairs()
symbols = [item['symbol'] for item in pairs if 'symbol' in item and item['symbol'].endswith('IDR') and item['symbol'] != 'BTCIDR']
remove_list = ['BTCIDR', 'USDTIDR', 'USDCIDR', 'TUSDIDR', 'WBTCIDR', 'DAIIDR']
symbols = [item for item in symbols if item not in remove_list]
print(symbols)

['1INCHIDR', 'AAVEIDR', 'AAIDR', 'AAZIDR', 'ABBCIDR', 'ABIIDR', 'ABYSSIDR', 'ACHIDR', 'ACTIDR', 'ACTSOLIDR', 'ADAIDR', 'ADPIDR', 'AEROIDR', 'AEVOIDR', 'AGIIDR', 'AIIDR', 'AIOZIDR', 'AIXBTIDR', 'ALICEIDR', 'ALIFIDR', 'ALGOIDR', 'ALPACAIDR', 'ALTIDR', 'ALITASIDR', 'AMPIDR', 'ANDYIDR', 'ANIMEIDR', 'ANKRIDR', 'AOAIDR', 'APEIDR', 'API3IDR', 'APUIDR', 'ARBIDR', 'ARKMIDR', 'ASIXV2IDR', 'ATHIDR', 'ATOMIDR', 'ATTIDR', 'AUCTIONIDR', 'AUDIOIDR', 'AVAXIDR', 'AXLIDR', 'AXSIDR', 'BADGERIDR', 'BAKEIDR', 'BALIDR', 'BANANAIDR', 'BANDIDR', 'BATIDR', 'BCDIDR', 'BCHIDR', 'BEAMIDR', 'BETAIDR', 'BGBIDR', 'BICOIDR', 'BIOIDR', 'BLURIDR', 'BNBIDR', 'FORMIDR', 'BOMEIDR', 'BONDIDR', 'BONEIDR', 'BOTXIDR', 'BRETTIDR', 'BSVIDR', 'BTRIDR', 'CAKEIDR', 'CATIDR', 'CATIIDR', 'CBGIDR', 'CELIDR', 'CELOIDR', 'CELRIDR', 'CGPTIDR', 'CHEEMSIDR', 'CHILLGUYIDR', 'CHRIDR', 'CHTIDR', 'CHZIDR', 'CINDIDR', 'CKBIDR', 'CNGIDR', 'C98IDR', 'CLVIDR', 'COLIDR', 'COMPIDR', 'CFXIDR', 'CONXIDR', 'COTIIDR', 'CREAMIDR', 'CROIDR', 'CRVIDR', 'C

In [14]:
# pairs = get_pairs()
# symbols = [item['symbol'] for item in pairs if 'symbol' in item and item['symbol'].endswith('USDT') and item['symbol'] != 'BTCUSDT']
# print(symbols)

In [15]:
now = datetime.now()
past = now - timedelta(days=1000)
timeFrom = int(time.mktime(past.timetuple()))
timeTo = int(time.mktime(now.timetuple()))
tf = '1D'
width_ratio = 0.2

In [16]:
dfBtcIdr = get_df_from_ticker('BTC', tf, timeFrom, timeTo)

In [17]:
def show_chart_rsi(df) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['RSI_14'], color='blue')
    plt.axhline(70, linestyle='--', alpha=0.5, color='red', label='Overbought (70)')
    plt.axhline(30, linestyle='--', alpha=0.5, color='green', label='Oversold (30)')
    plt.title('Nilai RSI Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('RSI')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'RSI-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [18]:
def show_chart_rsi_custom(df) :
    width = width_ratio * 2 * len(df['Label'])
    plt.figure(figsize=(width, 6))

    overbought = 70
    oversold = 30

    plt.fill_between(df.index, overbought, 100, color='red', alpha=0.3)
    plt.fill_between(df.index, 0, oversold, color='green', alpha=0.3)

    for i, row in df.iterrows():
        color = 'green' if row['RSI_14'] < oversold else 'red' if row['RSI_14'] > overbought else 'gray'
        plt.scatter(i, row['RSI_14'], color=color, label=row['Label'] if i == 0 else "")  # Show label only for first occurrence

    for i, row in df.iterrows():
        plt.text(i, row['RSI_14'], row['Label'], fontsize=7, ha='center', color='black')

    # Menambahkan label dan judul
    plt.title('RSI Indicator')
    plt.xlabel('Ticker')
    plt.ylabel('RSI Value')

    # plt.show()
    plt.savefig(os.path.join(directory, 'RSI-Custom-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [19]:
def show_chart_beta(df) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Beta'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Beta = 0')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Beta = 1')
    plt.title('Nilai Beta (Volatilitas Harga Token Terhadap BTC)')
    plt.xlabel('Ticker')
    plt.ylabel('Beta')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Beta-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [20]:
def show_chart_correl(df) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Correl'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='magenta', label='Correlation = 0')
    plt.axhline(-1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = -1')
    plt.axhline(1, linestyle='--', alpha=0.5, color='cyan', label='Correlation = 1')
    plt.title('Nilai Pearson Correlation Coefficient Terhadap BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Correlation')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Correlation-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [21]:
def calculate_downside_std(returns) :
    downside_returns = returns[returns < 0]
    if len(downside_returns) < 2 :
        return None
    return np.sqrt((downside_returns ** 2).sum()/(len(downside_returns) - 1))

In [22]:
def show_multiple_chart(tickers, show_individual_chart=False) :
    rsi_list = []
    beta_list = []
    correl_list = []
    latest_change_1d_list = []
    latest_change_7d_list = []
    latest_change_30d_list = []
    # return_mean_list = []
    # return_std_list = []
    sharpe_ratio_list = []
    adj_return_per_volatility_list = []
    sortino_ratio_list = []

    tickers.sort()

    for i, ticker in enumerate(tickers) :
        print('Processing ' + ticker + '...')
        will_show = False
        description = ''
        df_coin_idr_1 = get_df_from_ticker(ticker, tf, timeFrom, timeTo)
        if df_coin_idr_1.empty :
            continue
        df_btc_pair = get_history_btc_pair(ticker, df_coin_idr_1, dfBtcIdr)
        if show_individual_chart :
            show_chart(df_btc_pair, ticker, 'BTC')

        if 'RSI_14' in df_btc_pair.columns :
            latest_rsi = df_btc_pair['RSI_14'].iloc[-1]
            if not pd.isna(latest_rsi) :
                rsi_list.append({'Label': ticker, 'RSI_14': latest_rsi})
            if latest_rsi < 30 :
                will_show = True
                description = 'Low RSI'
            if latest_rsi > 70 :
                will_show = True
                description = 'High RSI'

        latest_beta = df_btc_pair['Beta'].iloc[-1]
        beta_list.append({'Label': ticker, 'Beta': latest_beta})

        if latest_beta < 0 :
            will_show = True
            description = 'Negative Beta'

        latest_correl = df_btc_pair['Correl'].iloc[-1]
        correl_list.append({'Label': ticker, 'Correl': latest_correl})

        if latest_correl < 0 :
            will_show = True
            description = 'Negative Correlation'

        # if will_show and not show_individual_chart :
        #     print(description)
        #     show_chart(df_btc_pair, ticker, 'BTC')

        latest_change_1d = df_btc_pair['Change1D'].iloc[-1]
        latest_change_7d = df_btc_pair['Change7D'].iloc[-1]
        latest_change_30d = df_btc_pair['Change30D'].iloc[-1]

        if not pd.isna(latest_change_1d) :
            latest_change_1d_list.append({'Label': ticker, 'Data': latest_change_1d})
        if not pd.isna(latest_change_7d) :
            latest_change_7d_list.append({'Label': ticker, 'Data': latest_change_7d})
        if not pd.isna(latest_change_30d) :
            latest_change_30d_list.append({'Label': ticker, 'Data': latest_change_30d})

        return_mean = df_btc_pair['Return'].mean(skipna=True)
        return_std = df_btc_pair['Return'].std(skipna=True)
        sharpe_ratio = return_mean/return_std

        latest_cum_return = df_btc_pair['Cum Return'].iloc[-1]
        return_downside_std = calculate_downside_std(df_btc_pair['Return'])
        adj_return_per_volatility = latest_cum_return / return_std

        # if not pd.isna(return_mean) :
        #     return_mean_list.append({'Label': ticker, 'Data': return_mean*100})
        # if not pd.isna(return_std) :
        #     return_std_list.append({'Label': ticker, 'Data': return_std})
        if not pd.isna(sharpe_ratio) :
            sharpe_ratio_list.append({'Label': ticker, 'Data': sharpe_ratio})
        if not pd.isna(adj_return_per_volatility) :
            adj_return_per_volatility_list.append({'Label': ticker, 'Data': adj_return_per_volatility})
        if not pd.isna(return_downside_std) :
            sortino_ratio = return_mean/return_downside_std
            sortino_ratio_list.append({'Label': ticker, 'Data': sortino_ratio})

    rsi_list = sorted(rsi_list, key=lambda x: x['RSI_14'])
    rsi_df = pd.DataFrame(rsi_list)
    show_chart_rsi(rsi_df)
    # show_chart_rsi_custom(rsi_df)

    beta_list = sorted(beta_list, key=lambda x: x['Beta'])
    beta_df = pd.DataFrame(beta_list)
    show_chart_beta(beta_df)

    correl_list = sorted(correl_list, key=lambda x: x['Correl'])
    corr_df = pd.DataFrame(correl_list)
    show_chart_correl(corr_df)

    latest_change_1d_list = sorted(latest_change_1d_list, key=lambda x: x['Data'])
    latest_change_1d_list_df = pd.DataFrame(latest_change_1d_list)
    show_chart_change(latest_change_1d_list_df, 'Perubahan 1 Hari Terhadap BTC')

    latest_change_7d_list = sorted(latest_change_7d_list, key=lambda x: x['Data'])
    latest_change_7d_list_df = pd.DataFrame(latest_change_7d_list)
    show_chart_change(latest_change_7d_list_df, 'Perubahan 7 Hari Terhadap BTC')

    latest_change_30d_list = sorted(latest_change_30d_list, key=lambda x: x['Data'])
    latest_change_30d_list_df = pd.DataFrame(latest_change_30d_list)
    show_chart_change(latest_change_30d_list_df, 'Perubahan 30 Hari Terhadap BTC')

    # return_mean_list = sorted(return_mean_list, key=lambda x: x['Data'])
    # return_mean_list_df = pd.DataFrame(return_mean_list)
    # show_chart_return_mean(return_mean_list_df)
    #
    # return_std_list = sorted(return_std_list, key=lambda x: x['Data'])
    # return_std_list_df = pd.DataFrame(return_std_list)
    # show_chart_return_std(return_std_list_df)

    sharpe_ratio_list = sorted(sharpe_ratio_list, key=lambda x: x['Data'])
    sharpe_ratio_list_df = pd.DataFrame(sharpe_ratio_list)
    show_chart_sharpe_ratio(sharpe_ratio_list_df)

    adj_return_per_volatility_list = sorted(adj_return_per_volatility_list, key=lambda x: x['Data'])
    adj_return_per_volatility_list_df = pd.DataFrame(adj_return_per_volatility_list)
    show_chart_adj_return_per_volatility(adj_return_per_volatility_list_df)

    sortino_ratio_list = sorted(sortino_ratio_list, key=lambda x: x['Data'])
    sortino_ratio_list_df = pd.DataFrame(sortino_ratio_list)
    show_chart_sortino_ratio(sortino_ratio_list_df)

    excel_file = os.path.join(directory, 'price-' + get_time_stamp() + '.xlsx')
    with pd.ExcelWriter(excel_file) as writer:
        rsi_df.to_excel(writer, sheet_name='RSI', index=False)
        beta_df.to_excel(writer, sheet_name='Beta', index=False)
        corr_df.to_excel(writer, sheet_name='Correlation', index=False)
        latest_change_1d_list_df.to_excel(writer, sheet_name='Change 1D', index=False)
        latest_change_7d_list_df.to_excel(writer, sheet_name='Change 7D', index=False)
        latest_change_30d_list_df.to_excel(writer, sheet_name='Change 30D', index=False)
        # return_mean_list_df.to_excel(writer, sheet_name='Return Mean', index=False)
        # return_std_list_df.to_excel(writer, sheet_name='Volatility', index=False)
        sharpe_ratio_list_df.to_excel(writer, sheet_name='Sharpe Ratio', index=False)
        sortino_ratio_list_df.to_excel(writer, sheet_name='Sortino Ratio', index=False)
        adj_return_per_volatility_list_df.to_excel(writer, sheet_name='Adj Return Per Volatility', index=False)

In [23]:
def show_chart_change(df, chart_title) :
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='Change = 0%')
    plt.title(chart_title)
    plt.xlabel('Ticker')
    plt.ylabel('Perubahan (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, chart_title + '-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [24]:
def show_chart_return_mean(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Rata-Rata Return Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Return Mean (%)')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Rata-Rata Return-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [25]:
def show_chart_return_std(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Volatilitas Return Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Tingkat Volatilitas')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Volatilitas-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [26]:
def show_chart_sharpe_ratio(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Sharpe Ratio Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Sharpe Ratio')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Sharpe Ratio-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [27]:
def show_chart_adj_return_per_volatility(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Adjusted Return Per Unit Volatility Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Adjusted Return Per Unit Volatility')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Adjusted Return Per Unit Volatility-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [28]:
def show_chart_sortino_ratio(df) :
    if 'Label' not in df.columns:
        return
    width = width_ratio * len(df['Label'])
    plt.figure(figsize=(width, 6))
    plt.scatter(df['Label'], df['Data'], color='blue')
    plt.axhline(0, linestyle='--', alpha=0.5, color='red', label='0')
    plt.title('Sortino Ratio Pair BTC')
    plt.xlabel('Ticker')
    plt.ylabel('Sortino Ratio')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.text(0.5, 0.5, 'Generated by Yoga',
         transform=plt.gca().transAxes,
         fontsize=40, color='gray', alpha=0.3,
         ha='center', va='center', rotation=30)
    plt.text(0.99, 1.05, datetime.now().strftime('%d-%m-%Y %H:%M:%S'),
         transform=plt.gca().transAxes,
         fontsize=12, color='gray', alpha=0.7,
         ha='right', va='bottom')
    # plt.show()
    plt.savefig(os.path.join(directory, 'Sortino Ratio-' + get_time_stamp() + '.png'), dpi=300, bbox_inches='tight')
    plt.close()

In [29]:
# tickers = [ 'FARTCOIN', 'MOODENG', 'SPX', 'ZEREBRO', 'WIF', 'ACTSOL', 'LADYS', 'ANDY', 'APU', 'BOME', 'BONE', 'BRETT', 'CAT', 'CATI', 'CHILLGUY', 'DEGEN', 'DOGE', 'DOGS', 'FLOKI', 'FWOG', 'GIGA', 'GOAT', 'MYRO', 'MEW', 'MOG', 'PEPE', 'NPC', 'PEOPLE', 'PONKE', 'POPCAT', 'SHIB', 'SLERF', 'SUNDOG', 'TURBO', 'PNUT', 'MEME', 'HMSTR', 'NEIRO', 'VIRTUAL', 'AERO', 'ALTLAYER', 'API3', 'ARB', 'ARKM', 'ATH', 'AVAX', 'BGB', 'BNB', 'CAKE', 'CTC', 'CRV', 'CTK', 'EIGEN', 'ENA', 'ENS', 'ETH', 'FET', 'FIL', 'FLUX', 'FTM', 'GALA', 'GMT', 'GRASS', 'GRT', 'HBAR', 'HIFI', 'HNT', 'IMX', 'IO', 'ISLM', 'JASMY', 'JUP', 'JTO', 'KMNO', 'LDO', 'LPT', 'LQTY', 'MBOX', 'ME', 'METIS', 'MKR', 'MNT', 'MOVE', 'NEAR', 'OM', 'ONDO', 'PAXG', 'PENDLE', 'PHA', 'PYTH', 'POL', 'RAY', 'REZ', 'RENDER', 'SOL', 'STORJ', 'SUI', 'TON', 'TRX', 'UNI', 'WORMHOLE', 'WLD', 'XRP', 'YGG','ZKJ', 'ZETA', 'VANRY', 'TRB', 'TOKEN', 'TNSR', 'TIA', 'TAIKO', 'SUSHI', 'SUN', 'STRK', 'SNX', 'SNT', 'SAFE', 'SAND', 'MANA', 'RSR', 'RARE', 'QNT', 'PUFFER', 'PRIME', 'POWR', 'PORTAL', 'PIXEL', 'OMNI', 'OP', 'NFP', 'MAVIA', 'MASK', 'LUNA', 'LUNC', 'LTC', 'LOOM', 'L3', 'LISTA', 'LINK', 'KSM', 'HIGH', 'ICP', 'ILV', 'GRAVITY', 'GMX', 'GLM', 'ETHFI', 'DUSK', 'DYDX', 'DOT', 'CTSI', 'CFX', 'COMP', 'CGPT', 'BEAM', 'BAND', 'BCH', 'AUCTION', 'AXS', 'ATOM', 'ACH', 'ADA', 'AI', 'TEN', 'ENJ', 'EOS', 'AEVO', 'ALICE', 'APE', 'BLUR', 'STG', 'JOE', 'INJ', '1INCH','AAVE', 'FORM', 'ID', 'XDC', 'SUPER', 'XLM']

# tickers = ['GIGA', 'POPCAT', 'ETH', 'SOL', 'BNB', 'MOG', 'WLD', 'APU', 'FWOG']

tickers = [symbol.removesuffix('IDR') for symbol in symbols]

show_multiple_chart(tickers, True)

Processing APU...
Processing BNB...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


Processing ETH...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


Processing FWOG...
Processing GIGA...
Processing MOG...
Processing POPCAT...
Processing SOL...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


Processing WLD...


/Users/yogabprawira/PycharmProjects/crypto-price/.venv/lib/python3.13/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


In [30]:
def show_chain_comparison_chart(chains) :
    full_chain_list = {}

    for i, chain in enumerate(chains) :
        print('Fetching ' + chain + '...')
        df_coin_idr = get_df_from_ticker(chain, tf, timeFrom, timeTo)
        full_chain_list[chain] = df_coin_idr

    for i, chain in enumerate(chains):
        for j, chain_base in enumerate(chains[:i]):
            print('Processing ' + chain + '/' + chain_base + '...')
            df_coin_idr = full_chain_list[chain]
            df_base_idr = full_chain_list[chain_base]
            df = get_history_btc_pair(chain, df_coin_idr, df_base_idr)
            show_chart(df, chain, chain_base)


In [31]:
chain_tickers = ['ETH', 'XRP', 'BNB', 'SOL', 'ADA', 'TRX', 'SUI', 'AVAX', 'TON', 'HBAR', 'DOT', 'NEAR', 'ICP', 'POL', 'ATOM', 'ARB', 'FTM']

show_chain_comparison_chart(chain_tickers)